# Fine tuning Llama 2 LORA

### Roadmap
Para concluir:
- Testar outros datasets
- Arrumar load do treino e inference na mesma função
- Continuar treino salvo
- Criar loop para treino de vários arquivos em uma pasta, já que não consigo carregar um dataset muito grande (dataset voto-ementa com 200k estoura a memória).

Concluído:
- Baixar modelo 7b
- Criar 7b HF
- Abrir 7b HF e rodar modelo
- Treinar em base teste
- Criar base assuntos
- Arrumar problema do SafeTensor
- Conferir template de entrada do llama em português


### Para configuração do ambiente
pip install -U pip

pip install tensorflow

pip install git+https://github.com/Keith-Hon/bitsandbytes-windows.git

pip install accelerate

pip install appdirs==1.4.4

pip install datasets

pip install fire==0.5.0

pip install sentencepiece==0.1.97

pip install peft

pip install tensorboardX==2.6

pip install gradio==3.23.0

pip install seaborn

In [ ]:
import transformers
import textwrap
import text_preprocessing
import re
from transformers import (LlamaTokenizer, 
                          LlamaForCausalLM, 
                          pipeline,
                          GenerationConfig,
                          AutoModelForCausalLM,
                          AutoTokenizer)
from transformers.utils import is_accelerate_available, is_bitsandbytes_available
import os
import sys
from typing import List
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)
import fire
import torch
from datasets import load_dataset
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams
from peft import PeftModel
from bs4 import BeautifulSoup
%matplotlib inline

sns.set(rc={'figure.figsize':(10, 7)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)



#--------------------------------------------Arguments------------------------------------
TEST_WITH_SMALLER_DATASET = False
LOAD_PREVIOUS_TRAINING = False
RESUME_FROM_CHECKPOINT = True
MODEL = 'llama-3-8b'
DATASET = 'voto_ementa'   #'resultado_fila472_2'
#--------------------------------------------Arguments------------------------------------
CUTOFF_LEN = 4096

if MODEL == 'llama-2-7b-hf':  #Versão em inglês
    MODEL_BASE = "./llama-2-7b-hf"
    TOKENIZER = "./llama-2-7b-hf"
    MY_MODEL_SUFFIX = 'base'
    CHECKPOINT_DIFF_FROM_BASE = False
elif MODEL == 'cabrita':
    MODEL_BASE = "decapoda-research/llama-7b-hf"
    TOKENIZER = "decapoda-research/llama-7b-hf"
    MY_MODEL_SUFFIX = 'loradecapodacabrita'
    CHECKPOINT_DIFF_FROM_BASE = True
    MODEL_LORA = "22h/cabrita-lora-v0-1"
elif MODEL == 'cabrita-baffo32':
    MODEL_BASE = "baffo32/decapoda-research-llama-7B-hf"
    TOKENIZER = "baffo32/decapoda-research-llama-7B-hf"
    MY_MODEL_SUFFIX = 'loradecapodaresearchcabrita'
    CHECKPOINT_DIFF_FROM_BASE = True
    MODEL_LORA = "22h/cabrita-lora-v0-1"
elif MODEL == 'llama-2-7b-chat-hf':
    MODEL_BASE = "./llama-2-7b-chat-hf"
    TOKENIZER = "./llama-2-7b-chat-hf"
    MY_MODEL_SUFFIX = 'chat'
    CHECKPOINT_DIFF_FROM_BASE = False
elif MODEL == 'llama-3-8b':
    MODEL_BASE = "./Meta-Llama-3-8B"
    TOKENIZER = "./Meta-Llama-3-8B"
    MY_MODEL_SUFFIX = 'Llama38B_v2'
    CHECKPOINT_DIFF_FROM_BASE = False
    #https://github.com/meta-llama/llama3/blob/main/README.md
    
if DATASET == 'resultado_fila472_2':
    DATASET_FILE = './datasets/resultado_fila472_2_dataset.json'
    MY_MODEL = 'checkpoints_resultado_fila472_2_3' + MY_MODEL_SUFFIX
elif DATASET == 'assuntos':
    DATASET_FILE = './datasets/dataset-llama-sentencas-assuntos.json'
    MY_MODEL = 'checkpoints_assuntos3_' + MY_MODEL_SUFFIX
elif DATASET =='completar_sentenca':
    DATASET_FILE = './datasets/dataset_completar_sentença.json'
    MY_MODEL = 'checkpoints_completar_sentenca_' + MY_MODEL_SUFFIX
elif DATASET =='voto_ementa':
    DATASET_FILE = './Datasets/Rel-Voto-Ementa_SemS3/dataset_voto_ementa_balanced.json'
    MY_MODEL = 'checkpoints_voto_ementa_' + MY_MODEL_SUFFIX
print('Creating model:', MY_MODEL)
print('Using Dataset:', DATASET_FILE)



#-------------------------------------- FORMATO DO PROMPT ------------------------------------------------
def generate_prompt_pt(data_point):
    return f"""Abaixo está uma instrução que descreve uma tarefa e uma entrada que fornece contexto adicional.
Escreva uma resposta que complete adequadamente a solicitação.
### Instrução:
{data_point["instruction"]}
### Entrada:
{data_point["input"]}
### Resposta:
{data_point["output"]}"""

def generate_prompt_eng(data_point):
    return f"""Below is an instruction that describes a task, 
paired with an input that provides further context. 
Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""

def template(inputstr:str, instructionstr:str):
    json_data = {'instruction': instructionstr,
                'input': inputstr,
                'output': ''}
    return generate_prompt_pt(json_data)

#------------Atualizar remover_citação_de_julgado e clean_voto com o utilizado em "Criar Dataset para Llama-------"
def remover_citação_de_julgado(texto):
    # Ainda falta testar bastante este algoritmo
    texto = re.sub('[\-ºA-Z\.,À-ÿ0-9 \\\/]{50,300}.{20,3000}\(.{20,100}[0-9\.]{6,20}.{20,100}\)', ' ', texto)
    return 
def clean_voto(texto):
    # Remover cabeçalho e outras coisas antes do voto
    texto = BeautifulSoup(texto, "lxml").text.replace('\xa0',' ')
    if 'V O T O' in texto:
        texto = re.split('V O T O', texto)[1]
    # Remover O exmo Juiz Fulano de Tal: no começo do voto
    texto = re.sub('^[OA].{10,140}:', '', texto.strip())
    texto = re.sub('(\n {0, 4})+', '\n', texto.strip())
    texto = remover_citação_de_julgado(texto)
    return texto



def carregar_dataset(dataset_file:str):
    print(dataset_file)
    data = load_dataset("json", data_files = dataset_file)
    print(data["train"])

    if TEST_WITH_SMALLER_DATASET:
        test_size = len(data["train"]) - 512
    else:
        test_size = 128   # Não preciso de muito Dataset de teste porque vou ter que testar na mão

    train_val = data["train"].train_test_split(test_size = test_size, 
                                               shuffle = True, 
                                               seed = 42)
    train_data = (train_val["train"].map(generate_and_tokenize_prompt))

    if TEST_WITH_SMALLER_DATASET:
        # Para limitar a quantidade de dados no treino, vou usar o train_test_split e us
        smaller_val_data = train_val["test"].train_test_split(test_size = 128, 
                                                              shuffle = True, 
                                                              seed = 42)
        val_data = (smaller_val_data["test"].map(generate_and_tokenize_prompt))
    else:
        val_data = (train_val["test"].map(generate_and_tokenize_prompt))
    
    print('train_data size:', len(train_data))
    print('val_data size:', len(val_data))
    return train_data, val_data, train_val

def test_inference(dataset, pipe, start:int=1, end:int=50):
    for i in range(start, end):
        example = dataset[i]
        input_template = template(example['input'], example['instruction'])
        output_template = example['output']
        print('INPUT TEMPLATE')
        print(input_template)
        print('-------------------------------------------------')
        print('DESIRED OUTPUT:')
        print(output_template)
        print('-------------------------------------------------')
        generated_text = pipe(input_template)
        print('GENERATED TEXT:')
        print(generated_text[0]['generated_text'].split('### Resposta:')[1])
        print('-------------------------------------------------')
        print('-------------------------------------------------')
        print('-------------------------------------------------')
        print('-------------------------------------------------')
        print('-------------------------------------------------')
        print('-------------------------------------------------')

def load_tokenizer_base(base_model_name:str, padding_side:str = 'right'):
    #tokenizer = LlamaTokenizer.from_pretrained(base_model_name)
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.pad_token_id = 0
    tokenizer.bos_token_id = 1
    tokenizer.eos_token_id = 2
    tokenizer.padding_side = padding_side
    return tokenizer

def load_hf_model(
    base_model_name: str,
    lora_model_name: str,
    device: str = "cuda",
    load_in_8bit: bool = True
):
    """
    Código baseado em https://github.com/tloen/alpaca-lora/blob/main/generate.py
    """

    if device == "cuda":
        if not is_accelerate_available():
            raise ValueError("Install `accelerate`")
    if load_in_8bit and not is_bitsandbytes_available():
        raise ValueError("Install `bitsandbytes`")

    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        load_in_8bit = load_in_8bit,
        torch_dtype = torch.float16,
        device_map = "auto",
    )
    model = PeftModel.from_pretrained(
        model,
        lora_model_name,
        torch_dtype = torch.float16,
    )

    model.config.pad_token_id = 0
    model.config.bos_token_id = 1
    model.config.eos_token_id = 2

    if not load_in_8bit:
        model.half()  # seems to fix bugs for some users.
    
    model.print_trainable_parameters()
    return model

if not any(not c.isalnum() for c in MY_MODEL):
    raise ValueError('Para o pytorch, não pode haver caracteres especiais (caso contrário ocorre erro no salvamento).')
    
torch.cuda.empty_cache()

In [ ]:
def old_tokenize(prompt:str, 
             add_eos_token = True, 
             max_length = CUTOFF_LEN,
             label:str = ''):
    result = tokenizer(
        prompt,
        truncation = True,
        max_length = max_length,
        padding = False,
        return_tensors = None,
    )
    IGNORE_INDEX = -100  # The default setting in CrossEntropyLoss
    
    if (result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < max_length
        and add_eos_token):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
    
    tokenized_label = tokenizer(label,
                                truncation = True,
                                max_length = max_length,
                                padding = False,
                                return_tensors = None)['input_ids']

    if (tokenized_label != tokenizer.eos_token_id
        and len(tokenized_label) < max_length
        and add_eos_token):
        tokenized_label.append(tokenizer.eos_token_id)
        
    #Esse é o erro: !!!!!
    #result["labels"] = result["input_ids"].copy()
    result["labels"] = tokenized_label
    
    return result

def tokenize(prompt:str,
             label:str = ''):
    # https://github.com/meta-llama/llama-recipes/blob/main/src/llama_recipes/datasets/grammar_dataset/grammar_dataset.py
    IGNORE_INDEX = -100  # The default setting in CrossEntropyLoss
    prompt_ids = tokenizer.encode(tokenizer.bos_token + prompt, add_special_tokens = False)
    label_ids = tokenizer.encode(label + tokenizer.eos_token, add_special_tokens = False)

    example = prompt_ids + label_ids
    return {"input_ids": example,
            "attention_mask": [1] * len(example),
            "labels": [IGNORE_INDEX] * len(prompt_ids) + label_ids}

def generate_and_tokenize_prompt(data_point):
    full_input_prompt = generate_prompt_pt(data_point)
    tokenized_full_prompt = tokenize(prompt = full_input_prompt, 
                                     label = data_point['output'])
    return tokenized_full_prompt

### Estou tendo dificuldade com o formato do dataset
1 - Tentei usar o código do llama_recipes (https://github.com/meta-llama/llama-recipes/blob/main/src/llama_recipes/datasets/grammar_dataset/grammar_dataset.py)
- Não funcionou
- GENERATED TEXT: Crie uma ementa para o seguinte voto: Crie uma ementa para o seguinte voto:#Crie uma ementa para o seguinte voto:#Crie uma ementa para o seguinte voto:#Crie uma ementa para o seguinte voto:#Crie uma ementa para o seguinte voto:#Crie uma ementa para o seguinte voto:#Crie uma ementa para o seguinte voto:#Crie uma ementa para o seguinte voto:#Cri


## Criando modelo Llama

In [ ]:
tokenizer = load_tokenizer_base(base_model_name = TOKENIZER)

if LOAD_PREVIOUS_TRAINING:
    model  = load_hf_model(base_model_name = MODEL_BASE,
                        lora_model_name = MY_MODEL + '_peft',
                        device = "cuda",
                        load_in_8bit = True)
else:
    model = LlamaForCausalLM.from_pretrained(MODEL_BASE,
                                            load_in_8bit=True,
                                            torch_dtype=torch.float16,
                                            device_map="auto")

    if CHECKPOINT_DIFF_FROM_BASE:
        model = PeftModel.from_pretrained(model, MODEL_LORA)

### LoRA

In [ ]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r = LORA_R,
    lora_alpha = LORA_ALPHA,
    target_modules = LORA_TARGET_MODULES,
    lora_dropout = LORA_DROPOUT,
    bias = "none",
    task_type = "CAUSAL_LM",
    inference_mode = False,
)

'''
Tentando continuar treinamento. Bug no treino.
AssertionError: No inf checks were recorded for this optimizer.

if LOAD_PREVIOUS_TRAINING:
    model = PeftModel.from_pretrained(
            model,
            MY_MODEL + '/checkpoint-200',
            config = config)
    print('ADAPTERS from LORA loaded.')
else:
'''
model = get_peft_model(model, config)
model.print_trainable_parameters()
print('ADAPTERS from LORA created.')

## Carregando Dataset

model.save_pretrained(MY_MODEL + '_teste_save_pretrained')
model2 = LlamaForCausalLM.from_pretrained(
    MODEL_BASE,
    load_in_8bit=True,
    #torch_dtype=torch.float16,
    device_map="auto",
    #use_safetensors=False
)
print('Base Model loaded. Loading adapters from PEFT:')
model2 = PeftModel.from_pretrained(
            model2,
            MY_MODEL + '_teste_save_pretrained')

In [ ]:
train_data, val_data, train_val = carregar_dataset(DATASET_FILE)

In [ ]:
for k in train_val['train'][0]:
    print(k)
    print(train_val['train'][0][k])
    print()
    print('---------------------------------------------')
    print()

In [ ]:
for k, v in generate_and_tokenize_prompt(train_val['train'][0]).items():
    print(k)
    print(v[-10:])
    print()
    print('---------------------------------------------')
    print()

In [ ]:
# Imprimir exemplo do Dataset

for key in train_data[0]:
    print(key, ': ', train_data[0][key])
    print()
    print('----------------------------------------------------')
    print()

### Trainer

strategy:
The evaluation strategy to adopt during training. Possible values are:
- `"no"`: No evaluation is done during training.
- `"steps"`: Evaluation is done (and logged) every `steps`.
- `"epoch"`: Evaluation is done at the end of each epoch.

In [ ]:
TEST_WITH_SMALLER_DATASET

In [ ]:
BATCH_SIZE = 4 #32 #128
PER_DEVICE_BATCH_SIZE = 1 # É o que importa pra memória
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // PER_DEVICE_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = len(train_data)
EVAL_STEPS = 200
SAVE_STEPS = 200
WARMUP_STEPS = 30
LOGGING_STEPS = 10

training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size = PER_DEVICE_BATCH_SIZE,
    per_device_eval_batch_size = PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS,
    warmup_steps = WARMUP_STEPS,
    max_steps = TRAIN_STEPS,
    learning_rate = LEARNING_RATE,
    fp16 = True, #Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    logging_steps = LOGGING_STEPS,
    optim = "adamw_torch",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = EVAL_STEPS,
    save_steps = SAVE_STEPS,
    output_dir = MY_MODEL,
    save_total_limit = 3,
    load_best_model_at_end = True,
    report_to = "tensorboard"
)
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, 
    pad_to_multiple_of = 8, 
    return_tensors = "pt", 
    padding = True
)
print('Training batch size = PER_DEVICE_TRAIN_BATCH_SIZE * n_gpu.')
print('Como n_gpu = ' + str(training_arguments.n_gpu) + '; Batch_size = ' + str(training_arguments.train_batch_size) + '.')
print('Eval batch size = ' + str(training_arguments.eval_batch_size) + '.')
print()
print(PER_DEVICE_BATCH_SIZE, 'PER_DEVICE_BATCH_SIZE')
print(GRADIENT_ACCUMULATION_STEPS, 'GRADIENT_ACCUMULATION_STEPS')

- gradient_checkpointing (`bool`, *optional*, defaults to `False`):
            If True, use gradient checkpointing to save memory at the expense of slower backward pass.
- auto_find_batch_size (`bool`, *optional*, defaults to `False`)
            Whether to find a batch size that will fit into memory automatically through exponential decay, avoiding
            CUDA Out-of-Memory errors. Requires accelerate to be installed (`pip install accelerate`)
- dataloader_prefetch_factor (`int`, *optional*):
            Number of batches loaded in advance by each worker.
            2 means there will be a total of 2 * num_workers batches prefetched across all workers.
            

In [ ]:
trainer = transformers.Trainer(
    model = model,
    train_dataset = train_data,
    eval_dataset = val_data,
    args = training_arguments,
    data_collator = data_collator,
)
#trainer.args.save_safetensors=False
#trainer.args.torch_compile = False
model.config.use_cache = False
'''
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))
'''
#model = torch.compile(model)
 
trainer.train(resume_from_checkpoint = RESUME_FROM_CHECKPOINT)
model.save_pretrained(MY_MODEL + '_save_pretrained')
trainer.save_model(MY_MODEL + '_peft')
print('model saved to', MY_MODEL + '_peft')

Step	Training Loss	Validation Loss
200	0.000300	0.000522
400	0.000000	0.000083
600	0.000000	0.000123
800	0.000000	0.000149
1000	0.000100	0.000435

In [ ]:
MY_MODEL

### Testing

In [ ]:
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    framework = "pt",
)
test_inference(val_data, pipe, start=1, end=10)

In [ ]:
example = val_data[0]
input_template = template(example['input'], example['instruction'])
a = input_template
print(a)

### Erro 1 - RecursionError: maximum recursion depth exceeded

- Erro ocorreu no 51 de 300. Apaguei no dataset Tweets que começam com RT e reiniciei tudo.

- Muitos dizem que é problema do tokenizador. Talvez baixar um outro llama resolva. Talvez alterar as configurações do tokenizador.

- Outra hipótese é atualizar a biblioteca dos Transformers.

git clone https://github.com/huggingface/transformers.git
cd transformers
git checkout c612628045822f909020f7eb6784c79700813eda
pip install -e .

- Tambem seria possível:

import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(1500)

- RESOLVEU apagar os Tweets que começam com RT. É provavelmente um erro do Tokenizer, e começou a funcionar porque apaguei o dado que o tokenizer dava problema. Não sei identificar que tipo de dado o Tokenizer dá problema. Preciso ficar atento porque esse ERRO PODE VOLTAR.

- Erro se repetiu com dataset super reduzido e Tweets que começam com RT removidos. Eu tentei executar a célula com o compile, deu erro de Windows, comentei a linha e executei de novo. Aí o erro ocorreu no training. O erro ocorreu no mesmo lugar: 51. Lembrando que o salvamento deve ocorrer a cada 50 épocas, faz sentido que o erro tenha algo a ver com salvamento.
- RESOLVEU remover o compile e rodar a célula de treino uma única vez.
- Ocorreu o erro novamente independentemente do compile. Aparentemente, o erro ocorre ao executar duas vezes a célula de treinamento.

### Erro 2 - SafetensorError: Error while deserializing header: InvalidHeaderDeserialization
- Erro ocorre ao tentar abrir arquivos salvos com safetensors na linha "with safe_open(filename, framework="pt", device=device) as f:". No github da huggingface (https://github.com/huggingface/transformers/issues/27397) eles dizem que o erro ocorre porque o statedict está vazio. E está vazio por causa do model = torch.compile(model). Mas não estou usando compile.
- Posso tentar salvar sem usar safetensors, mas se o statedict estiver vazio, vou ter outro tipo de problema.
- Em "from_pretrained", coloquei "use_safetensors=False". NÃO FUNCIONOU; ele continuou usando safe_tensors.
- Desinstalei safetensors. Pela documentação, quando safetensors não está instalado é utilizado pickle. NÃO FUNCIONOU: biblioteca Transformers não importa quando safetensors não está instalado.
- Usar transformers instalado a partir do pip. Acredito que o transformers instalado a partir do site da hugging face utiliza safe tensors enquanto que o transformers instalado a partir do pip utiliza pickle. NÃO FUNCIONOU. Ambas as versões usam safe tensors.
- Analisando o arquivo C:\Users\fabio\.conda\envs\envgpu\lib\site-packages\peft\utils save_and_load.py, ele seta o use_safetensors baseado na existência ou não de arquivo safetensors na pasta de salvamento. E como comecei utilizando safetensors, sempre havia esse tipo de arquivo lá. Apaguei todos os arquivos da pasta e rodei novamente. NÃO FUNCIONOU.
- Encontrei o que eu queria em C:\Users\fabio\.conda\envs\envgpu\lib\site-packages\transformers\trainer.py. No Trainer, dentro do args, o save_safetensors estava verdadeiro. Apenas adicionei aqui no notebook a linha "**trainer.args.save_safetensors=False**". Funcionou. Agora não está mais usando safetensors.
- Preciso checar se o erro era de fato o statedict estar vazio. E, por estar vazio, ocorre incompatibilidade com o safetensors. Para isso, adicionei print(adapters_weights) após adapters_weights = torch.load(filename, map_location=torch.device(device)) em C:\Users\fabio\.conda\envs\envgpu\lib\site-packages\peft\utils save_and_load.py. De fato, há um ERRO aqui. Ele imprimiu {}.
- Preciso checar o que está apagando o peso dos modelos durante o treinamento.
- Tentei trainer.args.torch_compile = False, mas esse valor já era falso.
- Na hora que o Trainer vai salvar, o statedict é None. Então adicionei no C:\Users\fabio\.conda\envs\envgpu\lib\site-packages\transformers\trainer.py, na linha 2917:
- '#---------------------------------------------------------------------------------------------------
- '# Fabio - Adicionei essas linhas aqui porque statedict estava None e isso gerava erros
- if state_dict is None:
-     state_dict = self.model.state_dict()
- '#print('Trainer - state_dict:', state_dict) #state_dict=None
- '#---------------------------------------------------------------------------------------------------
- Não funcionou porque o save_pretrained usa o get_peft_model_state_dict (C:\Users\fabio\.conda\envs\envgpu\lib\site-packages\peft\utils\save_and_load.py) para recuperar o state_dict. No get_peft_model_state_dict tem um momento em que o state_dict é perdido.
- Parece que ele está perdendo o state_dict na linha "to_return = {k: v for k, v in to_return.items() if (("lora_" in k and adapter_name in k) or ("bias" in k))}". O objetivo dessa linha parece ser economizar memória e salvar apenas os pesos do Adaptador do LoRA, ao invés de salvar o modelo inteiro.
- Parece que sempre tem lora_ na key. Mas no loop, uma vez, todas as keys têm o adapter_name ("default") e outra não têm, sucessivamente. Provavelmente há duas chamadas diferentes do save_pretrained, em uma há um modelo com adaptador e outra com llama puro sem adaptador. Preciso identificar as duas vezes provocando erro de assert.
- Entendi o que está acontecendo. Apesar de não saber porque programaram assim. Quando a função get_peft_model_state_dict é chamada com o parâmetro state_dict==None ela encontra o state_dict usando "state_dict = model.state_dict()". Porém esse model.state_dict() chama a recursivamente a função get_peft_model_state_dict. Ocorre que a get_peft_model_state_dict, no final, exclui o nome dos adaptadores na linha "to_return = {k.replace(f".{adapter_name}", ""): v for k, v in to_return.items()}" e sem o nome dos adaptadores, quando ela for filtrar apenas os adaptadores na linha "to_return = {k: v for k, v in to_return.items() if (("lora_" in k and adapter_name in k) or ("bias" in k))}", obviamente, gerará vazio {}.
- O que vou fazer é garantir que a função get_peft_model_state_dict nunca seja chamada com state_dict vazio.
- Adicionei após elif self.args.should_save: na função save_model do trainer (C:\Users\fabio\.conda\envs\envgpu\lib\site-packages\transformers\trainer.py):
    - state_dict = self.model.state_dict() #Eu (Fabio) que adicionei isso aqui. Parece-me correto. Espero não estar gerando um problema para outras funções.
    - self._save(output_ dir, state_dict=state_dict) # Adicionei o , state_dict=state_dict
- Não funcionou porque quando roda uma segunda vez, o nome do adaptador já foi removido pela linha final do get_peft_model_state_dict. Quando rodo novamente get_peft_model_state_dict em uma outra época, ele não acha nenhum adaptador.
- Solução que tentei foi **comentar** a linha final (**to_return = {k.replace(f".{adapter_name}", ""): v for k, v in to_return.items()}**) do **get_peft_model_state_dict** (C:\Users\fabio\.conda\envs\envgpu\lib\site-packages\peft\utils\ **save_and_load.py**) que remove o nome do adaptador. Aparentemente FUNCIONOU. Porém, como não sei porque ela estava lá, não sei afirmar quais bugs isso pode gerar.
- Atualizei o PEFT, erro voltou, comentei apenas a linha acima, arrumou. Ocorre que há o erro de perda de treino no salvamento. Então não vou comentar essa linha, por ora.
- O erro não ocorre quando não uso:
    - old_state_dict = model.state_dict
        model.state_dict = (
            lambda self, *_, **__: get_peft_model_state_dict(
                self, old_state_dict()
            )
        ).__get__(model, type(model))
- Remover esse código corrigiu tanto o bug do título quanto o problema de perder o treinamento ao carregar um modelo salvo.

### Erro 3 - Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_CUDA_mm)
- Ocorre quando interrompo e tento executar novamente. Se limpar a memória e executar tudo do zero funciona.


### Erro 4 - Carregar modelo salvo perde o treinamento.
Na página do Hugging Face sobre o peft (https://huggingface.co/docs/transformers/main/peft):
- To save your trained ADAPTER and load it back:
- model.save_pretrained(save_dir)
- model = AutoModelForCausalLM.from_pretrained(save_dir)

- Tenho 3 saves:
    - um do treino (MY_MODEL)
    - um do model.save_pretrained(MY_MODEL + '_save_pretrained')
    - um do trainer.save_model(MY_MODEL + '_peft')
- Nenhum save funcionou.
- Tenho 3 formas de load:
    - model = AutoModelForCausalLM.from_pretrained(save_dir)
    - model = LlamaForCausalLM.from_pretrained(save_dir)
    - model2 = LlamaForCausalLM.from_pretrained(
                MODEL_BASE,
                load_in_8bit=True,
                #torch_dtype=torch.float16,
                device_map="auto",
                #use_safetensors=False
            )
            model2 = PeftModel.from_pretrained(
                        model2,
                        MY_MODEL)
    - from peft import set_peft_model_state_dict
        STATE_DICT = MY_MODEL + '_save_pretrained/adapter_model.bin'
        full_state_dict = torch.load(STATE_DICT)
        set_peft_model_state_dict(model, full_state_dict)
- Nenhum load funcinou. (carrega, mas perde o treinamento)
- Tentei atualizar a biblioteca transformers da 36.0 para a 36.2. Não funcionou
- Tentei atualizar a biblioteca PEFT, não consegui carregar arquivos salvos na biblioteca antiga. É um bom sinal.

- RESOLVIDO. Removi o código:
    - old_state_dict = model.state_dict
            model.state_dict = (
                lambda self, *_, **__: get_peft_model_state_dict(
                    self, old_state_dict()
                )
            ).__get__(model, type(model))

# Erro 5 - datasetgenerationerror: an error occurred while generating the dataset
- Erro de memória quando tentava carregar o dataset. Estranho é que quando dá load no json ele funciona, mas 

# Erro 6 - estouro de memória quando CUTOFF_LEN > 2000

OutOfMemoryError: CUDA out of memory. Tried to allocate 7.64 GiB. GPU 0 has a total capacty of 23.99 GiB of which 0 bytes is free. Of the allocated memory 26.24 GiB is allocated by PyTorch, and 8.48 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Preciso ver se diminuir o batch permite manter uma entrada maior.

- Llama 2 com 2k tokens - rodou
- Llama 3 com 2k tokens e 32 batch - não
- Llama 3 com 1,5k tokens e 32 batch - rodou
- Llama 3 com 2k tokens e 1 batch - não
- Resetei o micro e Llama 3 com 2k tokens e 1 batch funcionou (dataset pequeno de teste)
- Sem resetar de novo, Llama 3 com 3k tokens e 1 batch funcionou (dataset pequeno de teste)

Por ora, parece que o Llama 3 só roda com 1,5k tokens independente do batch_size. Não faz sentido. Eu estava configurando o batch_size errado, agora arrumei. Mesmo assim, no teste de antes eu setei PER_DEVICE_BATCH_SIZE = 1 e não rodou o 2k tokens. Preciso refazer o teste.


Com Cut_off_len = 8190, ocorre erro se PER_DEVICE_BATCH_SIZE e GRADIENT_ACCUMULATION_STEPS forem grandes. Com PER_DEVICE_BATCH_SIZE = 1 e GRADIENT_ACCUMULATION_STEPS = 1, rodou.

# Erro 7 - Warning: You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.





# Inference

In [ ]:
from typing import Optional, Any

ALPACA_TEMPLATE = (
    "Below is an instruction that describes a task, paired with an input that provides "
    "further context. Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
)

def load_adapted_hf_generation_pipeline(
                                model,
                                tokenizer,
                                temperature: float = 0,
                                top_p: float = 1.,
                                max_tokens: int = 4096,
                                batch_size: int = 16,
                                task: str = "text-generation",
                                generation_kwargs: Optional[dict] = None,
                            ):

    model.eval()

    generation_kwargs = generation_kwargs if generation_kwargs is not None else {}
    config = GenerationConfig(
        do_sample = True,
        temperature = temperature,
        max_new_tokens = max_tokens,
        top_p = top_p,
        pad_token_id = 0,
        **generation_kwargs,
    )
    pipe = pipeline(
        task,
        model = model,
        tokenizer = tokenizer,
        batch_size = batch_size,
        generation_config = config,
        framework = "pt",
    )

    return pipe

In [ ]:
checkpoint_version = '_peft'
checkpoint_version = '/checkpoint-400'
model  = load_hf_model(base_model_name = MODEL_BASE,
                        lora_model_name = MY_MODEL + checkpoint_version,
                        device = "cuda",
                        load_in_8bit = True)

tokenizer = load_tokenizer_base(base_model_name = MODEL_BASE)

pipe = load_adapted_hf_generation_pipeline(
                    model = model,
                    tokenizer = tokenizer,
                    temperature = 0.01,
                    top_p = 1.,
                    max_tokens = CUTOFF_LEN,
                    batch_size = 1,
                    generation_kwargs = None,
                )

train_data, val_data = carregar_dataset(DATASET_FILE)

### Testando em vários exemplos

In [ ]:
test_inference(val_data, pipe, start=10, end=20)

-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
INPUT TEMPLATE
Abaixo está uma instrução que descreve uma tarefa e uma entrada que fornece contexto adicional.
Escreva uma resposta que complete adequadamente a solicitação.
### Instrução:
Crie uma ementa para o seguinte voto:
### Entrada:
Concedo a gratuidade para a parte autora.Não verifico nos autos nenhuma nulidade processual notadamente no que pertine à produção de provas e observância do pleno contraditório e da ampla defesa.Quanto ao mérito, não vejo razões para afastar a decisão do juízo de origem, devendo a r. sentença ser mantida por seus próprios fundamentos.O recurso da parte autora ressalta a concessão administrativa do benefício por incapacidade temporária, após perícia realiza

-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
INPUT TEMPLATE
Abaixo está uma instrução que descreve uma tarefa e uma entrada que fornece contexto adicional.
Escreva uma resposta que complete adequadamente a solicitação.
### Instrução:
Crie uma ementa para o seguinte voto:
### Entrada:
“A matéria relativa ao pedido de suspensão da exigibilidade das contribuições a entidades terceiras, na parte em que excederem a base de cálculo de 20 (vinte) vezes o maior salário mínimo vigente no país, é objeto dos Resp nº 1.898.532/CE e REsp nº 1.905.870/PR, afetados ao rito dos recursos representativos de controvérsia, estando assim delimitada:“  salários mínimos é aplicável à apuração da base de cálculo de "contribuições parafiscais arrecadadas por c

“A matéria relativa ao pedido de suspensão da exigibilidade das contribuições a entidades terceiras, na parte em que excederem a base de cálculo de 20 (vinte) vezes o maior salário mínimo vigente no país, é objeto dos Resp nº 1.898.532/CE e REsp nº 1.905.870/PR, afetados ao rito dos recursos representativos de controvérsia, estando assim delimitada:“  salários mínimos é aplicável à apuração da base de cálculo de "contribuições parafiscais arrecadadas por conta de terceiros", nos termos do art. 4º da Lei n. 6.950/1981, com as alterações promovidas em seu texto pelos arts. 1º e 3º do Decreto-Lei n. 2.
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
INPUT TEMPLATE
Abaixo está uma instrução que descreve uma tarefa e uma entrada que fornece co

-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
INPUT TEMPLATE
Abaixo está uma instrução que descreve uma tarefa e uma entrada que fornece contexto adicional.
Escreva uma resposta que complete adequadamente a solicitação.
### Instrução:
Crie uma ementa para o seguinte voto:
### Entrada:
O julgamento dos presentes embargos de declaração far-se-á com espeque no artigo 1024, do Código de Processo Civil.Os embargos de declaração são cabíveis para corrigir erro material, contradição, obscuridade ou omissão do acórdão (artigo, 1022 do Código de Processo Civil).Por certo a norma processual concede à parte o direito de ter os fundamentos de seu pedido apreciados pelo julgador. Entretanto, falta-lhe razão ao pretender seja apreciada questão que já

-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
INPUT TEMPLATE
Abaixo está uma instrução que descreve uma tarefa e uma entrada que fornece contexto adicional.
Escreva uma resposta que complete adequadamente a solicitação.
### Instrução:
Crie uma ementa para o seguinte voto:
### Entrada:
O Juízo de origem indeferiu o pedido o pedido do autor com base nos seguintes fundamentos: “(...)Para a concessão de tutela antecipada de urgência, é necessário o preenchimento cumulativo de três principais requisitos: a) a probabilidade do direito; b) o perigo de dano ou o risco ao resultado útil do processo; e c) a reversibilidade da tutela, podendo se conceder tutela irreversível somente nos casos em que o pleito for extremamente relevante e o indeferim

-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
INPUT TEMPLATE
Abaixo está uma instrução que descreve uma tarefa e uma entrada que fornece contexto adicional.
Escreva uma resposta que complete adequadamente a solicitação.
### Instrução:
Crie uma ementa para o seguinte voto:
### Entrada:
Trata-se de mandando de segurança com o objetivo de determinar à autoridade impetrada que dê prosseguimento ao despacho aduaneiro dos bens importados objeto das Declarações de Importação  nºs 22/0302220-4, 22/0357067-8 e 22/011822-8, mediante a conferência documental e verificação das mercadorias, com a consequente liberação dos bens importados, caso não existam impeditivos. Aduziu a impetrante que as mercadorias estão no Aeroporto de Viracopos em Campinas

-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
INPUT TEMPLATE
Abaixo está uma instrução que descreve uma tarefa e uma entrada que fornece contexto adicional.
Escreva uma resposta que complete adequadamente a solicitação.
### Instrução:
Crie uma ementa para o seguinte voto:
### Entrada:
DESEMBARGADOR FEDERAL WILSON ZAUHY:
 
Peço vênia para divergir do e. Relator para o efeito de declarar a incompetência do c. Órgão Especial para o julgamento do presente conflito.
O art. 11, inciso II, parágrafo único, alínea “i“ do Regimento Interno estabelece competir ao Órgão Especial o julgamento de “questões incidentes em processos da competência das Seções ou das Turmas que lhe hajam sido submetidas, bem assim os conflitos de competência entre Relato


-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------


In [ ]:
test_inference(train_data, pipe, start = 1, end = 10)

### Testando em 1 exemplo

In [ ]:
example = {}
example['instruction'] = 'Como deve o Tribunal Regional Federal da Terceira região responder ao seguinte chamado:'
example['input'] = 'Como instalar o certificado digital para acessar o PJe?'
input_template = template(example['input'], example['instruction'])
generated_text = pipe(input_template)
print(generated_text[0]['generated_text'].split('### Resposta:')[1])